In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
!pip install keras-unet-collection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 803.0 kB/s eta 0:00:00a 0:00:01


In [3]:
Height = 320
Width = 320
from keras_unet_collection import models

In [4]:
u2net_model = models.u2net_2d((Height, Width, 3), n_labels=1, 
                        filter_num_down=[32, 64, 128, 256], filter_num_up=[32, 32, 64, 128], 
                        filter_mid_num_down=[16, 14, 32, 64], filter_mid_num_up=[8, 16, 32, 64], 
                        filter_4f_num=[256, 256], filter_4f_mid_num=[128, 128], 
                        activation='ReLU', output_activation='Sigmoid', 
                        batch_norm=True, pool=False, unpool=False, deep_supervision=False, name='u2net')

----------
The depth of u2net_2d = len(filter_num_down) + len(filter_4f_num) = 6


In [5]:
unet_model = models.unet_2d((Height, Width, 3), [64, 128, 256, 512, 1024], n_labels=1,
                      stack_num_down=2, stack_num_up=2,
                      activation='ReLU', output_activation='Sigmoid', 
                      batch_norm=True, pool='max', unpool='nearest', name='unet')

In [6]:
from deeplabv3 import deeplabv3_plus
deeplab_model = deeplabv3_plus((Height, Width, 3))

94781440/94765736 [==============================] - 0s 0us/step


In [7]:
u2net_model.load_weights('../input/dl-models/model_u2net.h5')
unet_model.load_weights('../input/dl-models/model_unet.h5')
deeplab_model.load_weights('../input/dl-models/model_deeplab.h5')

In [8]:
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

def shuffle_data(image, mask):
    image, mask = shuffle(image, mask, random_state=42)
    return image, mask
def data_loading(data_path, split=0.25):
    images = sorted(glob(os.path.join(data_path, "ISIC2018_Task1-2_Training_Input", "*.jpg")))
    masks = sorted(glob(os.path.join(data_path, "ISIC2018_Task1_Training_GroundTruth", "*.png")))

    split_ratio = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_ratio, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_ratio, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_ratio, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_ratio, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def load_image(img_path):
    img_path = img_path.decode()
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height))
    img = img/255.0
    img = img.astype(np.float32)
    return img

def load_mask(mask_path):
    mask_path = mask_path.decode()
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (Width, Height))
    mask = mask/255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def tf_parsing(image, mask):
    def _parse(image, mask):
        image = load_image(image)
        mask = load_mask(mask)
        return image, mask

    image, mask = tf.numpy_function(_parse, [image, mask], [tf.float32, tf.float32])
    image.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return image, mask

def TF_dataset(images, masks, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.map(tf_parsing)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(10)
    return dataset

In [9]:
dataset_path = "../input"
train_path = os.path.join(dataset_path, "isic2018-challenge-task1-data-segmentation")

(Xtrain, Ytrain), (Xvalid, Yvalid), (Xtest, Ytest) = data_loading(train_path)

print(f"Test: {len(Xtest)} - {len(Ytest)}")

test_dataset = TF_dataset(Xtest, Ytest, batch_size=1)

Test: 648 - 648


In [10]:
models = [unet_model, deeplab_model , u2net_model]

In [11]:
model_input = tf.keras.Input(shape=(Height, Width, 3))
models = [unet_model, deeplab_model , u2net_model]
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
avg_ensemble = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

In [12]:
class WeightedLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2,w3, **kwargs):
        super(WeightedLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1]+ self.w3 * inputs[2]

In [13]:
model_input = tf.keras.Input(shape=(Height, Width, 3))
models = [unet_model, deeplab_model , u2net_model]
model_outputs = [model(model_input) for model in models]
ensemble_output = WeightedLayer(0.2,0.95,0.1)(model_outputs)
weighted_ensemble = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

In [14]:
unet_pred = unet_model.predict(test_dataset)
deeplab_pred = deeplab_model.predict(test_dataset)
u2net_pred = u2net_model.predict(test_dataset)
avg_preds = avg_ensemble.predict(test_dataset)
weihted_preds = weighted_ensemble.predict(test_dataset)

In [15]:
test_y = np.concatenate([y for x, y in test_dataset], axis=0)

In [16]:
from tensorflow.keras.metrics import Recall, Precision
def iou(y_true, y_pred): 
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    iou_value = (intersection + 1e-15) / (union + 1e-15)
    iou_value= iou_value.astype(np.float32)
    return iou_value
smooth = 1e-15
def dice_coeff(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


In [17]:
print("TEST_IOU for UNET",iou(test_y, unet_pred))
print("TEST_IOU for Deeplab",iou(test_y, deeplab_pred))
print("TEST_IOU for U2NET",iou(test_y, u2net_pred))
print("TEST_IOU for average_ensemble",iou(test_y, avg_preds))
print("TEST_IOU for weighted_ensemble",iou(test_y, weihted_preds))

TEST_IOU for UNET 0.7384786
TEST_IOU for Deeplab 0.82535386
TEST_IOU for U2NET 0.5093771
TEST_IOU for average_ensemble 0.6796715
TEST_IOU for weighted_ensemble 0.95655113


In [18]:
print("TEST_DICE for UNET",dice_coeff(test_y, unet_pred).numpy())
print("TEST_DICE for Deeplab",dice_coeff(test_y, deeplab_pred).numpy())
print("TEST_DICE for U2NET",dice_coeff(test_y, u2net_pred).numpy())
print("TEST_DICE for average_ensemble",dice_coeff(test_y, avg_preds).numpy())
print("TEST_DICE for weighted_ensemble",dice_coeff(test_y, weihted_preds).numpy())

TEST_DICE for UNET 0.849569
TEST_DICE for Deeplab 0.9043121
TEST_DICE for U2NET 0.6749498
TEST_DICE for average_ensemble 0.8092915
TEST_DICE for weighted_ensemble 0.97778946


In [19]:
cutoff = 0.5                              
y_pred_classes = np.zeros_like(weihted_preds)
y_pred_classes[weihted_preds > cutoff] = 1
                            
y_true_classes = np.zeros_like(test_y)
y_true_classes[test_y > cutoff] = 1

In [20]:
m = tf.keras.metrics.Recall()
m.update_state(test_y.flatten(),unet_pred.flatten())
print('TEST_Recall for UNET', m.result().numpy())

m.update_state(test_y.flatten(),deeplab_pred.flatten())
print('TEST_Recall for Deeplab', m.result().numpy())

m = tf.keras.metrics.Recall()
m.update_state(test_y.flatten(),u2net_pred.flatten())
print('TEST_Recall for U2net', m.result().numpy())

m = tf.keras.metrics.Recall()
m.update_state(test_y.flatten(),avg_preds.flatten())
print('TEST_Recall for Avg ensemble', m.result().numpy())

m = tf.keras.metrics.Recall()
m.update_state(y_true_classes.flatten(),y_pred_classes.flatten())
print('TEST_Recall for weighted ensemble', m.result().numpy())

TEST_Recall for UNET 0.81776196
TEST_Recall for Deeplab 0.8483259
TEST_Recall for U2net 0.8098698
TEST_Recall for Avg ensemble 0.84128666
TEST_Recall for weighted ensemble 0.8835119


In [21]:
m = tf.keras.metrics.Precision()
m.update_state(test_y.flatten(),unet_pred.flatten())
print('TEST_Precision for UNET', m.result().numpy())

m.update_state(test_y.flatten(),deeplab_pred.flatten())
print('TEST_Precision for Deeplab', m.result().numpy())

m = tf.keras.metrics.Precision()
m.update_state(test_y.flatten(),u2net_pred.flatten())
print('TEST_Precision for U2net', m.result().numpy())

m = tf.keras.metrics.Precision()
m.update_state(test_y.flatten(),avg_preds.flatten())
print('TEST_Precision for Avg ensemble', m.result().numpy())

m = tf.keras.metrics.Precision()
m.update_state(y_true_classes.flatten(),y_pred_classes.flatten())
print('TEST_Precision for weighted ensemble', m.result().numpy())

TEST_Precision for UNET 0.88612175
TEST_Precision for Deeplab 0.9100255
TEST_Precision for U2net 0.90373516
TEST_Precision for Avg ensemble 0.93549013
TEST_Precision for weighted ensemble 0.9296289


In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (Height, Width))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (Height, Width))
    ori_x = x
    x = x/np.max(x)
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((Height, 10, 3)) * 255.0

    ori_y = np.expand_dims(ori_y, axis=-1) 
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) 
    y_pred = np.expand_dims(y_pred, axis=-1)   
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0 

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
make_dir("results")
SCORE = []
for x, y in tqdm(zip(Xtest, Ytest), total=len(Xtest)):
    name = x.split("/")[-1]

     
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

        
    y_pred = weighted_ensemble.predict(x)[0] > 0.5
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred.astype(np.int32)

        
    save_image_path = f"results/{name}"
    save_result(ori_x, ori_y, y_pred, save_image_path)

In [ ]:
import shutil
shutil.make_archive('./results', 'zip', 'results')

In [ ]:
from IPython.display import FileLink
FileLink(r'./results.zip')